# Issue 46: add functionality to choose minimizing or maximizing for each metric in a dataset

[See issue progress on GitHub here.](https://github.com/NREL/tyche/issues/46)

In [1]:
import os
import sys

sys.path.insert(1, os.path.abspath("../../src"))

import numpy             as np
import matplotlib.pyplot as pl
import pandas            as pd
import seaborn           as sb
import tyche             as ty

# Sustainable Aviation Fuel

In [2]:
saf_loc = "../../tutorial/data/saf"

saf_designs = ty.Designs(saf_loc)
saf_investments = ty.Investments(saf_loc)
saf_designs.compile()
tranche_results = saf_investments.evaluate_tranches(saf_designs,
                                                    sample_count=1000)

techs = tranche_results.summary.index.get_level_values('Technology').unique()

Evaluating HEFA Camelina
Evaluating HEFA Castor
Evaluating HEFA Jatropha
Evaluating HEFA Pennycress
Evaluating HEFA Yellow Grease


## Using the `opt_slsqp` (formerly `min_slsqp`) method

Evaluate the maximum metric values using the `optimum_metrics` method (formerly the `max_metrics` method). After refactoring, the method can take either a `Dict` of optimization senses, one per metric, or a `str` specifying one optimization sense to be used across all metrics. If no sense is provided, then the optimization sense used to instantiate the `EpsilonConstraintOptimizer` class object is used for all metrics.

First demonstrate the use of an optimization sense dictionary, with one sense per metric.

In [ ]:
sense_dict = {'Reduction in MJSP' : 'max',
              'Reduction in Jet GHG': 'max',
              'MJSP': 'min',
              'Jet GHG': 'min'}

opt_metric_values = {}

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
    )
    opt_metric_values[i] = optimizer.optimum_metrics(
        sense = sense_dict,
        total_amount = 5000000
    )

opt_metric_values

Next, provide a sense (max) as a string to `optimum_metrics` that overrides the sense provided to the `EpsilonConstraintOptimizer` object.

In [ ]:
max_metric_values = {}

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator,
    )
    max_metric_values[i] = optimizer.optimum_metrics(
        sense = 'max',
        total_amount = 5000000
    )

max_metric_values

Optimize for minimum jet selling price (MJSP) by minimizing MJSP directly.

In [ ]:
min_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "MJSP",
        sense = 'min',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    min_results.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_results

Now optimize for MJSP by maximizing the reduction in MSJP from the state of technology value. This will determine if the same solutions are found when minimizing MJSP versus maximizing the reduction in MJSP.

In [ ]:
max_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "Reduction in MJSP",
        sense = 'max',
        total_amount = 5000000        
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    max_results.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, max_dict, w

max_results

The MJSP values match when MJSP is minimized or when the reduction in MJSP is maximized.

Now perform the same test on the GHG metrics. First, minimize jet GHG.

In [ ]:
min_ghg = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "Jet GHG",
        sense = 'min',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    min_ghg.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_ghg

Now, maximize the reduction in jet GHG.

In [ ]:
max_red_ghg = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "Reduction in Jet GHG",
        sense = 'max',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    max_red_ghg.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, max_dict, w

max_red_ghg

These optimal values match as well when jet GHG is minimized versus when the reduction in jet GHG is maximized.

## Using the `opt_milp` (formerly `pwlinear_milp`) method

In [ ]:
min_mjsp = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_milp(
        "MJSP",
        sense = 'min',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    min_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    min_mjsp.append(
        min_dict
    )
    del tranche_tech_results, evaluator, optimizer, min_dict, w

min_mjsp

In [ ]:
max_red_mjsp = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_milp(
        "Reduction in MJSP",
        sense = 'max',
        total_amount = 5000000
    )
    print(f"{i} - {w[1]}")
    max_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    max_red_mjsp.append(
        max_dict
    )
    del tranche_tech_results, evaluator, optimizer, max_dict, w

max_red_mjsp

## Epsilon constraint formulation using optimization sense

All epsilon constraints are formulated as value - limit >= 0, hence LHS >= RHS. There is already a sense parameter in the method used to construct the "value" or LHS of the constraint, that applies a negative to the value when the sense is "maximum" and does not apply a negative when the sense is "minimum". (Actual parameter values are 'min' and 'max' respectively.) 

To construct a lower-bound epsilon constraint, no negative is applied to the "value" and the constraint is formulated as value - limit >= 0 --> value >= limit. This happens when the `eps_sense` parameter on the optimization method (`opt_slsqp` or `opt_milp`) is set to 'min'.

To construct an upper-bound epsilon constraint, the constraint is formulated as -1 * value - limit >= 0 --> value <= limit. This happens when `eps_sense` is set to 'max'.

`eps_sense` defaults to 'min', thus by default epsilon constraints are constructed as lower bounds on the metrics.

Demonstrate this functionality by minimizing MJSP (sense = 'min') while constraining the reduction in jet GHG emissions to be greater than zero (eps_sense = 'min').

In [ ]:
i = techs[0]

tranche_tech_results = tranche_results.summary[
    tranche_results.summary.index.get_level_values('Technology') == i
]
evaluator = ty.Evaluator(
    saf_investments.tranches, tranche_tech_results
)
optimizer = ty.EpsilonConstraintOptimizer(
    evaluator
)
w = optimizer.opt_slsqp(
    "MJSP",
    sense = 'min',
    total_amount = 5000000,
    eps_metric = pd.Series([0.0], name="Value", index = ["Reduction in Jet GHG"])
)

In [ ]:
eps_results = []

for i in techs:
    tranche_tech_results = tranche_results.summary[
        tranche_results.summary.index.get_level_values('Technology') == i
    ]
    evaluator = ty.Evaluator(
        saf_investments.tranches, tranche_tech_results
    )
    optimizer = ty.EpsilonConstraintOptimizer(
        evaluator
    )
    w = optimizer.opt_slsqp(
        "MJSP",
        sense = 'min',
        total_amount = 5000000,
        eps_metric = pd.Series([0.0], name="Value", index = ["Reduction in Jet GHG"]),
        eps_sense = pd.Series(['min'], name="Value", index = ["Reduction in Jet GHG"])
    )
    print(f"{i} - {w[1]}")
    eps_dict = {
        'Technology': i,
        'Optimizer status': w[1],
        'Investment amounts': w[2],
        'Optimum metrics': w[3]
    }
    eps_results.append(
        eps_dict
    )
    del tranche_tech_results, evaluator, optimizer, eps_dict, w

eps_results

The reduction in jet GHG values in the optimal solutions are within the tolerance value.